In [1]:
import sys
import time
import numpy as np
import pickle
import importlib

In [2]:
from qiskit import Aer, IBMQ
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [4]:
max_size = 57
max_length = 56

In [5]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [6]:
import libs_qrem
importlib.reload(libs_qrem)
from libs_qrem import QREM_Filter_4

In [7]:
xz_lnp_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    xz_lnp_mitigator_list.append(QREM_Filter_4(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of xz_lnp_mitigator_list: ", len(xz_lnp_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
length of xz_lnp_mitigator_list:  56


In [8]:
zx_lnp_mitigator_list = []
for n in range(2, max_size + 1): # [2, ..., 57]
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 2], mit_pattern=mit_pattern)
    zx_lnp_mitigator_list.append(QREM_Filter_4(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of zx_lnp_mitigator_list: ", len(zx_lnp_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
length of zx_lnp_mitigator_list:  56


In [9]:
lnp_hist_list = []
xz_lnp_total_times = []
zx_lnp_total_times = []
for i in range(max_length):
    print(i + 2, "qubits")
    
    t1 = time.perf_counter()
    lnp_hist_list.append( xz_lnp_mitigator_list[i].apply(raw_hist_list[2 * i]) )
    t2 = time.perf_counter()
    xz_lnp_total_times.append(t2 - t1)
    print("xz", t2 - t1, "s")
    
    t3 = time.perf_counter()
    lnp_hist_list.append( zx_lnp_mitigator_list[i].apply(raw_hist_list[2 * i + 1]) )
    t4 = time.perf_counter()
    zx_lnp_total_times.append(t4 - t3)
    print("zx", t4 - t3, "s")
    
    print()

2 qubits
xz 0.0001007380000004332 s
zx 2.80620000125964e-05 s

3accumulator is positive, we might even ignoring the necessal positive values.
 qubits
xz 3.647100000137016e-05 s
zxaccumulator is positive, we might even ignoring the necessal positive values.
 4.8152000005075024e-05 s

4 qubits
xz 0.00014461399999277091 s
zx 7.148900000686353e-05 s

5 qubits
xz 0.00011254799998994258 s
zx 0.00012063200000511642 s

6 qubits
xz 0.0002186339999923348 s
zx 0.00022384000000386095 s

7 qubits
xz 0.0004838330000040969 s
zx 0.0006373499999909882 s

8 qubits
xz 0.001169509999996876 s
zx 0.00131048000000078 s

9 qubits
xz 0.0022363249999983736 s
zx 0.0034944050000120797 s

10 qubits
xz 0.005645935000003988 s
zx 0.006010172999992847 s

11 qubits
xz 0.008055827000006843 s
zx 0.018307976999992093 s

12 qubits
xz 0.026486847000001035 s
zx 0.02806950599999425 s

13 qubits
xz 0.02892841400000634 s
zx 0.06523304099999905 s

14 qubits
xz 0.0811074460000043 s
zx 0.08338493299999072 s

15 qubits
xz 0.0907577

In [10]:
with open("./pkls/lnp_hist_list.pkl", "wb") as f:
    pickle.dump(lnp_hist_list, f)

In [11]:
with open("./pkls/lnp_total_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_total_times, "zx": zx_lnp_total_times}, f)

In [13]:
xz_lnp_times = []
zx_lnp_times = []
for i in range(max_length):
    xz_lnp_times.append( xz_lnp_mitigator_list[i].times() )
    zx_lnp_times.append( zx_lnp_mitigator_list[i].times() )

In [14]:
with open("./pkls/lnp_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_times, "zx": zx_lnp_times}, f)

In [33]:
xz_lnp_mitigators = []
for i in range(max_length):
    xz_lnp_mitigators.append({"indices_to_keys_vector":xz_lnp_mitigator_list[i].indices_to_keys_vector(),
                              # "mitigated_hist":xz_lnp_mitigator_list[i].mitigated_hist(),
                              "one_norm":xz_lnp_mitigator_list[i].one_norm(),
                              # "reduced_inv_A":xz_lnp_mitigator_list[i].reduced_inv_A(),
                              "sum_of_x":xz_lnp_mitigator_list[i].sum_of_x(),
                              "sum_of_x_hat":xz_lnp_mitigator_list[i].sum_of_x_hat(),
                              "sum_of_x_tilde":xz_lnp_mitigator_list[i].sum_of_x_tilde(),
                              # "times":xz_lnp_mitigator_list[i].times(),
                              "x_s":xz_lnp_mitigator_list[i].x_s(),
                              "x_hat":xz_lnp_mitigator_list[i].x_hat(),
                              "x_tilde":xz_lnp_mitigator_list[i].x_tilde(),
                              })
    print(i, "finished")

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished
26 finished
27 finished
28 finished
29 finished
30 finished
31 finished
32 finished
33 finished
34 finished
35 finished
36 finished
37 finished
38 finished
39 finished
40 finished
41 finished
42 finished
43 finished
44 finished
45 finished
46 finished
47 finished
48 finished
49 finished
50 finished
51 finished
52 finished
53 finished
54 finished
55 finished


In [34]:
zx_lnp_mitigators = []
for i in range(max_length):
    zx_lnp_mitigators.append({"indices_to_keys_vector":zx_lnp_mitigator_list[i].indices_to_keys_vector(),
                              # "mitigated_hist":zx_lnp_mitigator_list[i].mitigated_hist(),
                              "one_norm":zx_lnp_mitigator_list[i].one_norm(),
                              # "reduced_inv_A":zx_lnp_mitigator_list[i].reduced_inv_A(),
                              "sum_of_x":zx_lnp_mitigator_list[i].sum_of_x(),
                              "sum_of_x_hat":zx_lnp_mitigator_list[i].sum_of_x_hat(),
                              "sum_of_x_tilde":zx_lnp_mitigator_list[i].sum_of_x_tilde(),
                              # "times":zx_lnp_mitigator_list[i].times(),
                              "x_s":zx_lnp_mitigator_list[i].x_s(),
                              "x_hat":zx_lnp_mitigator_list[i].x_hat(),
                              "x_tilde":zx_lnp_mitigator_list[i].x_tilde(),
                              })
    print(i, "finished")

0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished
25 finished
26 finished
27 finished
28 finished
29 finished
30 finished
31 finished
32 finished
33 finished
34 finished
35 finished
36 finished
37 finished
38 finished
39 finished
40 finished
41 finished
42 finished
43 finished
44 finished
45 finished
46 finished
47 finished
48 finished
49 finished
50 finished
51 finished
52 finished
53 finished
54 finished
55 finished


In [35]:
with open("./pkls/lnp_mitigator_list.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_mitigators, "zx": zx_lnp_mitigators}, f)